# Libraries

In [64]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import cv2
import tensorflow as tf
from tensorflow.keras.applications import vgg16
import zipfile
import os
from sklearn.model_selection import train_test_split
import keras.backend as K
from sklearn.metrics import log_loss

# Data

In [65]:
train_unzip= zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/train.zip')
train_unzip.extractall()

In [66]:
train_unzip= zipfile.ZipFile('../input/dogs-vs-cats-redux-kernels-edition/test.zip')
train_unzip.extractall()

In [67]:
train=os.listdir("./train")
test =os.listdir("./test")

In [68]:
def lab(k):
    return(k.split('.')[0])

train=pd.DataFrame(train,columns=['path'])
train['label']=train['path'].apply(lab)

In [69]:
train['label'].loc[train.label=='cat']=0
train['label'].loc[train.label=='dog']=1

In [70]:
train.head()

# Data Split

In [71]:
x_train,x_test=train_test_split(train['path'],test_size=0.20)
x_train,x_cv  =train_test_split(x_train,test_size=0.18)

# Data Generator

In [72]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,x_in, batch_size=32,n_classes=2):
        self.batch_size = batch_size
        self.x_in = x_in
        self.n_classes = n_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.x_in.shape[0] / self.batch_size))

    def __getitem__(self, index):
        x, y = self.__data_generation(self.x_in[index*self.batch_size:(index+1)*self.batch_size])
        return x, y

    def __data_generation(self, ini):
        x = np.empty((self.batch_size,224,224,3))
        y = np.empty((self.batch_size))
        
        for i in range(ini.shape[0]):
            t   = cv2.imread("./train/"+str(ini.values[i]))
            x[i]= cv2.resize(t,(224,224))
            if ini.values[i].split('.')[0]=='dog':
                y[i] = 1
            else:
                y[i] = 0
        return x, y

train_generator=DataGenerator(x_train)
cv_generator   =DataGenerator(x_cv)

In [73]:
class Test_DataGenerator(tf.keras.utils.Sequence):
    def __init__(self,x_in, batch_size=1,n_classes=2):
        self.batch_size = batch_size
        self.x_in = x_in
        self.n_classes = n_classes
        self.on_epoch_end()

    def __len__(self):
        return int(np.floor(self.x_in.shape[0] / self.batch_size))

    def __getitem__(self, index):
        x = self.__data_generation(self.x_in[index*self.batch_size:(index+1)*self.batch_size])
        return x

    def __data_generation(self, ini):
        x = np.empty((self.batch_size,224,224,3))
        for i in range(ini.shape[0]):
            t   = cv2.imread("./train/"+str(ini.values[i]))
            x[i]= cv2.resize(t,(224,224))
        return x

test_generator=Test_DataGenerator(x_test)

# Model

In [74]:
base_model = vgg16.VGG16(weights='../input/keras-pretrained-models/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5',include_top=False,input_shape=(224,224, 3))
for layer in base_model.layers:
    layer.trainable = False

In [75]:
x = tf.keras.layers.Flatten()(base_model.output)
x = tf.keras.layers.Dense(256,activation='relu')(x)
x = tf.keras.layers.Dense(1,activation='sigmoid')(x)

model = tf.keras.Model(inputs=base_model.input, outputs=x)
model.summary()

In [76]:
class CustomCallBacks(tf.keras.callbacks.Callback):
    
    def on_train_begin(self, logs={}):
        self.loss     ={'loss':[]}
        self.val_loss ={'loss':[]}
        self.epoch    =None
        self.b        =0
        self.c        =0
        self.min_t    =None 
        
    def on_epoch_begin(self, epoch, logs={}):
        self.epoch=epoch
        if self.epoch>=1:
            lr=tf.keras.backend.get_value(self.model.optimizer.lr)
            tf.keras.backend.set_value(self.model.optimizer.lr,lr*0.10)

    def on_epoch_end(self, epoch, logs={}):
        print('{} ended'.format(epoch))
        self.loss['loss'].append(logs.get('loss'))
        self.val_loss['loss'].append(logs.get('val_loss'))
        if self.epoch==0:
            self.model.save_weights('cats_dogs.h5')
            self.min_t=self.val_loss['loss'][-1]
        if self.epoch>=1:
            if self.val_loss['loss'][-1]<=self.min_t and self.val_loss['loss'][-1]<=self.loss['loss'][-1]:
                self.min_t=self.val_loss['loss'][-1]
                self.model.save_weights('cats_dogs.h5')
                print('*'*5)
                print("weight updated")
                print('*'*5)
cb=CustomCallBacks()

In [77]:
opt = tf.keras.optimizers.Adam(learning_rate=0.001)
model.compile(optimizer=opt,loss='binary_crossentropy',metrics=['accuracy'])

In [78]:
model.fit_generator(train_generator,validation_data=cv_generator,epochs=3,callbacks=[cb])

In [79]:
model.load_weights('./cats_dogs.h5')

In [80]:
y_pred=model.predict_generator(test_generator,verbose=1)

In [81]:
x_test.shape

In [82]:
label=[]
for i in x_test:
    if (i.split('.')[0])=='dog':
        label.append(1)
    else:
        label.append(0)

In [83]:
len(label)

In [84]:
len(y_pred)

In [85]:
print(y_pred.reshape(1,-1)[0])